# **ETL (Extracción, Transformación y Carga de los datos)**
#En este proceso vamos a realizar las transformaciones necesarias, con el fin de que nuestros datos queden optimizados para nuestras consultas.

###   Antes de iniciar, deberemos importar las librerias necesarias que facilitaran nuestro proceso.

In [41]:
#Importamos las librerias
import pandas as pd
import numpy as np
import ast
import re
import json
import nltk

### Ahora es necesario cargar los datasets proporcionados para iniciar con nuestro proceso, en este caso nos vamos a valer de la función ***'read_csv'*** de la libreria Pandas, ya que concuerda con el formato en que fueron entregado los datos.

In [18]:
#Cargar los datos desde csv
movies = pd.read_csv('/content/movies_dataset.csv')
credits = pd.read_csv('/content/credits.csv')


<ipython-input-18-074ee4df6042>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('/content/movies_dataset.csv')


### Se evaluan las estructuras y dimensiones los datasets proporcionados.

In [19]:
#Observamos la estructura y distribución de los datos
print(movies.shape)
print(credits.shape)
print(movies.info())
print(credits.info())
print(movies.head(1))
print(credits.head(1))

(45466, 24)
(45476, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 no

### Para mayor facilidad de trabajo sería prudente unir ambos datasets, sin embargo la única columna similar es el '***id***', pero los tipos de datos son distintos, primero se transformará y luego se hará la unión.

In [20]:
#Transformamos la columna 'id' de Movies a valor numérico
movies['id'] = pd.to_numeric(movies['id'], errors='coerce')

#Hacemos la unión de los dos datasets con la función 'merge()' de Pandas.
movies = pd.merge(left=movies, right = credits, how='inner', on='id')
#'id' quedó como flatante, vamos a pasarlo a int
movies['id'] = movies['id'].astype(int)
#Revisaremos la nueva distribución de los datos
movies.shape

(45538, 26)

### Se revisan los valores columna por columna, para verificar el estado de los datos. Se concluye que hay columnas con valores que pueden ser importantes dentro de un sistema complejo de datos. Se infiere que el valor de '***name***' de la mayoria de las columnas puede ser valioso. Así como extraer el nombre de los actores y diectores de cada pelicula.
### Para ello se utilizará una clase llamada ***'magic'***, en esta clase almacenamos las principales funciones de extracción y transformación de datos que usaremos

In [21]:
class magic:
    '''Esta clase almacena funciones de transformación
    y extracción de datos, aducuada para cada situción'''
    @staticmethod
    def transform_str(valor):
      '''convierte el valor de entrada en
       una cadena. Si el valor es una
       lista o un diccionario, se convierte
       en una cadena JSON. De lo contrario,
       se convierte en una cadena normal .'''
      if isinstance(valor, (list, dict)):
        return json.dumps(valor)
      return str(valor)

    @staticmethod
    def extract_name(valor):
      '''extrae el valor del nombre de una
       cadena de entrada. La función utiliza
       una expresión regular para buscar el
       patrón 'name': '([^']*)' en la cadena
       de entrada. Si se encuentra una coincidencia,
       se devuelve el valor del nombre. sino devuelve None'''
      pattern = r"'name': '([^']*)'"
      matchpt = re.findall(pattern, valor)
      if len(matchpt) > 0:
        name = matchpt[0]
        return name
      else:
        return None

    def find_director(obj):
      '''busca el nombre del director en una
       cadena de entrada. Esta se convierte
       en una lista de diccionarios con la función
       ast.literal_eval(). Luego, busca el diccionario
       que tiene el trabajo 'Director' y devuelve el
       valor del nombre. Si no se encuentra un diccionario
       con el trabajo 'Director', se devuelve una lista vacía '''
      list_director = []
      for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
          list_director.append(i['name'])
          break
      return list_director


    def get_actor (valor):
      '''busca los nombres de los actores en una cadena de entrada.
       La cadena de entrada se convierte en una lista de diccionarios
       utilizando la función ast.literal_eval(). Luego, la función
       agrega el valor del nombre del actor a una lista. La función
       se detiene después de agregar los primeros nombres de actores
       a la lista.'''
      lista_cast = []
      counter = 0
      for i in ast.literal_eval(valor):
        if counter != 4:
          lista_cast.append(i['name'])
          counter += 1
        else:
          break
      return lista_cast

    def transform_date(df, column_name):
      '''Se utiliza una expresión regular para buscar el
       patrón "\d{4}-\d{2}-\d{2}" en la columna especificada.
       Si se encuentra una coincidencia, se devuelve la fecha.
       De lo contrario, se devuelve None.'''
      pattern = r"\d{4}-\d{2}-\d{2}"
      df[column_name] = df[column_name].apply(lambda x: x if re.match(pattern, str(x)) else None)
      return df[column_name]

    def get_genres(valor):
      '''busca los nombres de los géneros en una cadena de entrada.
      La cadena de entrada se convierte en una lista de diccionarios
      con la función ast.literal_eval(). Luego, la función agrega el
      valor del nombre del género a una lista.'''
      genres = [i['name'] for i in ast.literal_eval(valor)]
      return genres



### De algunas columnas se necesita tan solo un valor, pero en otras se requiere una lista de valores, por eso se aplican las funciones de la clase ***'magic'*** a cada columna por separado, dependiendo de la estructura.

In [22]:
#En esta celda se aplican las extracciones de los valores que se necesitan, de aquellas columnas con estructuras de datos complejas
movies['name_colection'] = movies['belongs_to_collection'].apply(magic.transform_str).apply(magic.extract_name)

movies['genres'] = movies['genres'].apply(magic.get_genres)

movies['production_companies'] = movies['production_companies'].apply(magic.transform_str).apply(magic.extract_name)

movies['production_countries'] = movies['production_countries'].apply(magic.transform_str).apply(magic.extract_name)

movies['spoken_languages'] = movies['spoken_languages'].apply(magic.transform_str).apply(magic.extract_name)

movies['actor'] = movies['cast'].apply(magic.get_actor)

movies['director'] = movies['crew'].apply(magic.find_director)

### El dataframe ha quedado con 29 columnas, algunas con datos redundantes, al final del proceso de transformación se eliminarán las que no sean necesarias.

In [7]:
movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,cast,crew,name_colection,actor,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",Toy Story Collection,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney]",[John Lasseter]


### En el enunciado se pide imputar el valor de ***0*** donde existan los valores nulos en las columnas ***'budget'*** y ***'revenue'***. Y posteriormente realizar una operación matemática entre ellos.

In [23]:
#Se verifican los campos con la cantidad de valores nulos
print(movies['revenue'].isnull().sum())
print(movies['budget'].isnull().sum())

3
0


In [25]:
#Se coloca el valor de 0 donde hay valores nulos con la función '.fillna()'
movies['revenue'].fillna(0, inplace=True)
movies['budget'].fillna(0, inplace=True)

### Tal y como es solicitado se procederá a crear la columna ***'return'***, dividiendo ***'revenue'*** / ***'budget'***, cuando no hay datos disponibles para calcularlo, deberá tomar el valor ***0***.

In [26]:
#Primero se verifican los tipos de datos, para saber si la operación puede ser factible
print(movies['revenue'].dtypes)
print(movies['budget'].dtypes)

float64
object


In [27]:
#Se transforma la columna budget a valor numérico
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce')

#Creamos la columna 'return'
movies['return'] = movies['revenue']/movies['budget']

#Donde no se haya podido realizar la operación se imputa el valor de 0
movies['return'].fillna(0, inplace=True)

### Ahora se transformará la columna de ***'release_date'*** para que tenga el formato AAAA-mm-dd.

In [28]:
#transformaremos los datos de de columna 'release_date' utilizando la función 'transform_date' de la clase creada magic
movies['release_date'] = magic.transform_date(movies, 'release_date')

#Eliminamos los valores faltantes de release_date
movies.dropna(subset=['release_date'],inplace=True)

### Se creará la columna ***'release_year'*** a partir de extraer el año de la columna ***'release_date'***. Pero antes se deberá transformar la columna a formato datetime.

In [29]:
#Aplicamos la transformación de la columna 'release_date'
movies['release_date'] = pd.to_datetime(movies['release_date'])

#Creamos la columna 'release_year'
movies['release_year'] = movies['release_date'].dt.strftime('%Y')

### A este punto se borrara las columnas que no serán necesarias para los siguientes pasos.

In [30]:
movies = movies.drop(columns=['video','imdb_id','adult','original_title','poster_path','homepage','belongs_to_collection','cast','crew', 'runtime','original_language','production_companies','production_countries','spoken_languages','name_colection',], axis=1)

In [31]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45451 entries, 0 to 45537
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   budget        45451 non-null  int64         
 1   genres        45451 non-null  object        
 2   id            45451 non-null  int64         
 3   overview      44510 non-null  object        
 4   popularity    45451 non-null  object        
 5   release_date  45451 non-null  datetime64[ns]
 6   revenue       45451 non-null  float64       
 7   status        45371 non-null  object        
 8   tagline       20425 non-null  object        
 9   title         45451 non-null  object        
 10  vote_average  45451 non-null  float64       
 11  vote_count    45451 non-null  float64       
 12  actor         45451 non-null  object        
 13  director      45451 non-null  object        
 14  return        45451 non-null  float64       
 15  release_year  45451 non-null  object

### A este punto se hace una revisión de los valores duplicados, considerando el valor de la columna ***'id'***, pueden haber películas con el titulos y mismos actores, pero deberían tener un numero de identificación distinto, si se tratan de distintas películas.

In [32]:
movies.loc[movies.duplicated(subset=['id'])].head(5)

,budget,genres,id,overview,popularity,release_date,revenue,status,tagline,title,vote_average,vote_count,actor,director,return,release_year
677,0,"[Drama, Romance]",105045,"East-Berlin, 1961, shortly after the erection ...",0.122178,1995-02-16,0.0,Released,"A love, a hope, a wall.",The Promise,5.0,1.0,"[Corinna Harfouch, Meret Becker, August Zirner...",[Margarethe von Trotta],0.0,1995
678,0,"[Drama, Romance]",105045,"East-Berlin, 1961, shortly after the erection ...",0.122178,1995-02-16,0.0,Released,"A love, a hope, a wall.",The Promise,5.0,1.0,"[Corinna Harfouch, Meret Becker, August Zirner...",[Margarethe von Trotta],0.0,1995
679,0,"[Drama, Romance]",105045,"East-Berlin, 1961, shortly after the erection ...",0.122178,1995-02-16,0.0,Released,"A love, a hope, a wall.",The Promise,5.0,1.0,"[Corinna Harfouch, Meret Becker, August Zirner...",[Margarethe von Trotta],0.0,1995
842,0,[Drama],132641,"Ten years into a marriage, the wife is disappo...",0.096079,1953-04-29,0.0,Released,NaN,Wife,0.0,0.0,"[Mieko Takamine, Ken Uehara, Rentarô Mikuni, M...",[Mikio Naruse],0.0,1953
843,0,[Drama],132641,"Ten years into a marriage, the wife is disappo...",0.619388,1953-04-29,0.0,Released,NaN,Wife,0.0,0.0,"[Mieko Takamine, Ken Uehara, Rentarô Mikuni, M...",[Mikio Naruse],0.0,1953


### En esta ocasión nos quedamos con el dataframe sin valores duplicados, reseteando el índice y borrando el anterior

In [17]:
#De esta manera el dataframe conservará solo los valores únicos, filtrado por 'id'
movies = movies.loc[~movies.duplicated(subset=['id'])]\
.reset_index(drop=True)

### Con estas 16 columnas el dataframe está casi listo para empezar a trabajar con él.

## Se Creará la columna ***'tag'***
### Corresponde fusionar las columnas con datos identificativos de cada pelicula y agregar esos valores en una sola columna, eso no solo reducirá la dimensión del dataframe sino que será importante al momento de crear el modelo de recomendación.
### Antes se deberá crear un dataframe provisorio con las columnas a utilizar

In [33]:
#Creamos un dataframe con las columnas que nutriran nuestra columna 'tag' mas adelante
mvs = movies[['id','title','overview','genres','tagline','actor','director']]
mvs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45451 entries, 0 to 45537
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        45451 non-null  int64 
 1   title     45451 non-null  object
 2   overview  44510 non-null  object
 3   genres    45451 non-null  object
 4   tagline   20425 non-null  object
 5   actor     45451 non-null  object
 6   director  45451 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.8+ MB


### Algunos nombres de actores pueden ser iguales, por ejemplo *'Tom Cruise'*, *'Tom Hanks'* y *'Tom Holland'*. Son actores distintos y los tre se llaman *'Tom'*, para diferenciarlos, retiramos el espacio, y así ya *'TomCruise'* y *'TomHolland'* serán distintos al momento de ser procesados.

In [34]:
#Con la función Lambda se recorre cada lista en busca de un espacio y une las palabras de cada columna.
mvs['genres'] = mvs['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
mvs['actor'] = mvs['actor'].apply(lambda x: [i.replace(" ", "") for i in x])
mvs['director'] = mvs['director'].apply(lambda x: [i.replace(" ", "") for i in x])

<ipython-input-34-1fa12ebf7eb3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mvs['genres'] = mvs['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
<ipython-input-34-1fa12ebf7eb3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mvs['actor'] = mvs['actor'].apply(lambda x: [i.replace(" ", "") for i in x])
<ipython-input-34-1fa12ebf7eb3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

### Para poder concatenar las columnas y crear la columna ***'tag'***, es importante considerar que tienen distintos tipos de datos, por lo que se procede a transformarlas todas a una cadena de texto. Luego se reemplazan los valores nulos por  una " ***,*** " de esta forma no afectará tanto cuando se necesite concatenar las columnas.

In [35]:
#Con la función 'applymap' transformamos todo los valores del dataframe a string.
mvs = mvs.applymap(str)
mvs.fillna(',', inplace=True)

### Una vez aplicadas las transformaciones necesarias, se puede proceder a concatenar todas las columnas y crear la columna ***'tag'***.

In [36]:
mvs['tag'] = mvs['overview'] + mvs['tagline'] + mvs['genres'] +  mvs['actor'] + mvs['director']
mvs['tag'][0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.nan['Animation', 'Comedy', 'Family']['TomHanks', 'TimAllen', 'DonRickles', 'JimVarney']['JohnLasseter']"

### Al crear la columna 'tag', quedan algunos simbolos que pueden causar problemas mas adelante, por lo cual se reemplaza, dejando el texto con cierta apariencia normal, pero con todos los valores allí. Así mismo por se transforman caracteres de Mayúscula a minúscula.

### Una vez que la columna 'tag' ha quedado lista, se puede agregar al dataframe original.

In [37]:
movies['tag'] = mvs[['tag']]
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45451 entries, 0 to 45537
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   budget        45451 non-null  int64         
 1   genres        45451 non-null  object        
 2   id            45451 non-null  int64         
 3   overview      44510 non-null  object        
 4   popularity    45451 non-null  object        
 5   release_date  45451 non-null  datetime64[ns]
 6   revenue       45451 non-null  float64       
 7   status        45371 non-null  object        
 8   tagline       20425 non-null  object        
 9   title         45451 non-null  object        
 10  vote_average  45451 non-null  float64       
 11  vote_count    45451 non-null  float64       
 12  actor         45451 non-null  object        
 13  director      45451 non-null  object        
 14  return        45451 non-null  float64       
 15  release_year  45451 non-null  object

### Ahora procedemos a extrar de la lista en el que se encontraban los valores de la columna ***'director'***.

In [38]:
movies['director'] = movies['director'].apply(lambda x: x[0] if len(x) > 0 else None)

### Ahora si se pueden eliminar todas las columnas que no tienen relevancia para el desarrollo de la API, dejando dejando el dataframe listo para ser trabajado.

In [39]:
movies = movies.drop(columns=['genres','status','tagline','overview'], axis=1)
movies.shape

(45451, 13)

### A partir de este momento se procederá a trabajar con la columna 'tag', para limpiar de manera mas eficiente, y dejarla lista para cuando correspoda su uso.

In [42]:
#A través del módulo nltk vamos a separar cada palabra por cada columna para realizar algunas transformaciones.
from nltk.tokenize import word_tokenize
nltk.download('punkt')
movies['tag'] = movies['tag'].apply(nltk.word_tokenize)
movies['tag'][0]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['Led',
 'by',
 'Woody',
 ',',
 'Andy',
 "'s",
 'toys',
 'live',
 'happily',
 'in',
 'his',
 'room',
 'until',
 'Andy',
 "'s",
 'birthday',
 'brings',
 'Buzz',
 'Lightyear',
 'onto',
 'the',
 'scene',
 '.',
 'Afraid',
 'of',
 'losing',
 'his',
 'place',
 'in',
 'Andy',
 "'s",
 'heart',
 ',',
 'Woody',
 'plots',
 'against',
 'Buzz',
 '.',
 'But',
 'when',
 'circumstances',
 'separate',
 'Buzz',
 'and',
 'Woody',
 'from',
 'their',
 'owner',
 ',',
 'the',
 'duo',
 'eventually',
 'learns',
 'to',
 'put',
 'aside',
 'their',
 'differences.nan',
 '[',
 "'Animation",
 "'",
 ',',
 "'Comedy",
 "'",
 ',',
 "'Family",
 "'",
 ']',
 '[',
 "'TomHanks",
 "'",
 ',',
 "'TimAllen",
 "'",
 ',',
 "'DonRickles",
 "'",
 ',',
 "'JimVarney",
 "'",
 ']',
 '[',
 "'JohnLasseter",
 "'",
 ']']

In [43]:
def remove_short_words(valor):
  token = []
  for word in valor:
    if len(word)>=3:
      token.append(word)
  return token

movies['tag'] = movies['tag'].apply(remove_short_words)


In [44]:
movies['tag'][0]

['Led',
 'Woody',
 'Andy',
 'toys',
 'live',
 'happily',
 'his',
 'room',
 'until',
 'Andy',
 'birthday',
 'brings',
 'Buzz',
 'Lightyear',
 'onto',
 'the',
 'scene',
 'Afraid',
 'losing',
 'his',
 'place',
 'Andy',
 'heart',
 'Woody',
 'plots',
 'against',
 'Buzz',
 'But',
 'when',
 'circumstances',
 'separate',
 'Buzz',
 'and',
 'Woody',
 'from',
 'their',
 'owner',
 'the',
 'duo',
 'eventually',
 'learns',
 'put',
 'aside',
 'their',
 'differences.nan',
 "'Animation",
 "'Comedy",
 "'Family",
 "'TomHanks",
 "'TimAllen",
 "'DonRickles",
 "'JimVarney",
 "'JohnLasseter"]

In [45]:
#Eliminamos ahora eliminamos las stopwords, es decir son palabras comunes que no aportan información relevante al momento de analizar texto.
from nltk.corpus import stopwords
nltk.download("stopwords")

def remove_stop_words(valor):
  a=set(stopwords.words('english'))
  token = [word for word in valor if word not in a]
  return token

movies['tag'] = movies['tag'].apply(remove_stop_words)
movies['tag'][0]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['Led',
 'Woody',
 'Andy',
 'toys',
 'live',
 'happily',
 'room',
 'Andy',
 'birthday',
 'brings',
 'Buzz',
 'Lightyear',
 'onto',
 'scene',
 'Afraid',
 'losing',
 'place',
 'Andy',
 'heart',
 'Woody',
 'plots',
 'Buzz',
 'But',
 'circumstances',
 'separate',
 'Buzz',
 'Woody',
 'owner',
 'duo',
 'eventually',
 'learns',
 'put',
 'aside',
 'differences.nan',
 "'Animation",
 "'Comedy",
 "'Family",
 "'TomHanks",
 "'TimAllen",
 "'DonRickles",
 "'JimVarney",
 "'JohnLasseter"]

In [46]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

def lemmatizer_words(valor):

  token = [WordNetLemmatizer().lemmatize(word) for word in valor]
  return token

movies['tag'] = movies['tag'].apply(lemmatizer_words)
movies['tag'][0]

[nltk_data] Downloading package wordnet to /root/nltk_data...


['Led',
 'Woody',
 'Andy',
 'toy',
 'live',
 'happily',
 'room',
 'Andy',
 'birthday',
 'brings',
 'Buzz',
 'Lightyear',
 'onto',
 'scene',
 'Afraid',
 'losing',
 'place',
 'Andy',
 'heart',
 'Woody',
 'plot',
 'Buzz',
 'But',
 'circumstance',
 'separate',
 'Buzz',
 'Woody',
 'owner',
 'duo',
 'eventually',
 'learns',
 'put',
 'aside',
 'differences.nan',
 "'Animation",
 "'Comedy",
 "'Family",
 "'TomHanks",
 "'TimAllen",
 "'DonRickles",
 "'JimVarney",
 "'JohnLasseter"]

In [47]:
#Se eliminamos algunos caracteres innecesarios resultantes de la transformación.
movies['tag'] = movies['tag'].apply(lambda x: [i.replace(".nan", "") for i in x])
movies['tag'] = movies['tag'].apply(lambda x: [i.replace(",", "") for i in x])

#convertimos de nuevo a un string todos los valores de la columna
movies['tag']= movies['tag'].apply(' '.join)
movies['tag'][0]

"Led Woody Andy toy live happily room Andy birthday brings Buzz Lightyear onto scene Afraid losing place Andy heart Woody plot Buzz But circumstance separate Buzz Woody owner duo eventually learns put aside differences 'Animation 'Comedy 'Family 'TomHanks 'TimAllen 'DonRickles 'JimVarney 'JohnLasseter"

###Para finalizar el proceso de ETL Se exporta el dataframe en el mismo formato en que fue entregado '.csv'

In [48]:
movies.to_csv('movies.csv', index=False)